In [1]:
from tensorflow import keras as keras
import tensorflow as tf
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_path = "dataset/train"
test_path = "dataset/test"

size = 300
img_size = (size, size)
n_classes = 5

datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest',
    validation_split=0.2
)

train = datagen.flow_from_directory(
    train_path,
    target_size = img_size,
    batch_size=64,
    seed = 42,
    subset='training',
    shuffle = True,
    class_mode='categorical'
)

validation = datagen.flow_from_directory(
    train_path,
    target_size = img_size,
    batch_size=64,
    seed = 42,
    subset='validation',
    shuffle = True,
    class_mode='categorical'
)

test = datagen.flow_from_directory(
    test_path,
    target_size = img_size,
    batch_size = 64,
    seed = 42,
    class_mode = 'categorical',
    shuffle = False
)


Found 4258 images belonging to 5 classes.
Found 1063 images belonging to 5 classes.
Found 1233 images belonging to 5 classes.


In [3]:
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras.applications import VGG16

In [4]:
model = keras.Sequential()
conv_base = VGG16(include_top=False, input_shape=(size, size, 3))
conv_base.trainable = False
"""
model.add(Conv2D(128, (1, 1), input_shape=(size, size, 3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), padding="same"))
model.add(Conv2D(256, (3, 3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), padding="same"))
model.add(Conv2D(256, (5, 5), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), padding="same"))
model.add(Conv2D(256, (7, 7), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2), padding="same"))
"""
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(0.5))
model.add(Dense(5, activation="softmax"))


In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 9, 9, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 41472)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               10617088  
_________________________________________________________________
dense_1 (Dense)              (None, 0)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 5         
Total params: 25,331,781
Trainable params: 10,617,093
Non-trainable params: 14,714,688
_________________________________________________________________


In [8]:
history = model.fit(train, batch_size=16, epochs=25, verbose=1, validation_data=validation)

Epoch 1/25


In [ ]:
losses = pd.DataFrame(model.history.history)
losses[['accuracy','val_accuracy']].plot()
losses[['loss','val_loss']].plot()